## Code for Attention and Slouching

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Load model
model_path = r"G:/zolo/classroom-attention-monitor//models/pose_model.pkl"
model = joblib.load(model_path)

# Class names
class_names = {0: 'Attentive', 1: 'Slouching'}

# MediaPipe pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb)

    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract features
        landmarks = result.pose_landmarks.landmark
        row = []
        for lm in landmarks:
            row.extend([lm.x, lm.y, lm.visibility])
        
        if len(row) == 99:  # 33 points × 3
            X = np.array(row).reshape(1, -1)
            pred = model.predict(X)[0]
            # Rule-based Raise Hand detection (Left and Right)

            label = class_names[int(pred)]
            cv2.putText(frame, f"Prediction: {label}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
    
    cv2.imshow("Pose Attention Monitor", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestCla

## Code for Hand raise

In [3]:
import cv2
import mediapipe as mp

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
drawing = mp.solutions.drawing_utils

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    label = ""

    if results.pose_landmarks:
        drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        lm = results.pose_landmarks.landmark

        # Get keypoints
        left_wrist = lm[15]
        left_shoulder = lm[11]
        right_wrist = lm[16]
        right_shoulder = lm[12]

        # Left hand raised
        if left_wrist.visibility > 0.5 and left_shoulder.visibility > 0.5:
            if left_wrist.y < left_shoulder.y:
                label += "Raise Hand (L) "

        # Right hand raised
        if right_wrist.visibility > 0.5 and right_shoulder.visibility > 0.5:
            if right_wrist.y < right_shoulder.y:
                label += "Raise Hand (R) "

    if not label:
        label = "Hands Down"

    cv2.putText(frame, label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 255), 3)
    cv2.imshow("Raise Hand Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Combined Code for Handraise and (Attention or Slouching)

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Load pose classification model
model_path = r"G:/zolo/classroom-attention-monitor/models/pose_model.pkl"
model = joblib.load(model_path)

# Class names from model
class_names = {0: 'Attentive', 1: 'Slouching'}

# MediaPipe pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb)

    full_label = ""

    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract features for pose model
        landmarks = result.pose_landmarks.landmark
        row = []
        for lm in landmarks:
            row.extend([lm.x, lm.y, lm.visibility])

        if len(row) == 99:
            X = np.array(row).reshape(1, -1)
            pred = model.predict(X)[0]
            pose_label = class_names.get(int(pred), "Unknown")
            full_label = pose_label

        # Rule-based Raise Hand detection
        left_wrist = landmarks[15]
        left_shoulder = landmarks[11]
        right_wrist = landmarks[16]
        right_shoulder = landmarks[12]

        if left_wrist.visibility > 0.5 and left_shoulder.visibility > 0.5:
            if left_wrist.y < left_shoulder.y:
                full_label += " + Raise Hand (L)"

        if right_wrist.visibility > 0.5 and right_shoulder.visibility > 0.5:
            if right_wrist.y < right_shoulder.y:
                full_label += " + Raise Hand (R)"
    else:
        full_label = "No Person Detected"

    # Display final combined label
    cv2.putText(frame, f"{full_label}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 255), 3)

    cv2.imshow("Pose Attention Monitor", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestCla